In [1]:
!pip install ogb

In [2]:
from torch_scatter import scatter_softmax, scatter_mean
from tqdm import tqdm

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Embedding, ModuleList
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU
from torch_scatter import scatter, scatter_mean, scatter_add, scatter_sum
from torch_geometric.nn import GINConv, GINEConv


class AtomEncoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(AtomEncoder, self).__init__()

        self.embeddings = torch.nn.ModuleList()

        for i in range(9):
            self.embeddings.append(Embedding(100, hidden_channels))

    def reset_parameters(self):
        for embedding in self.embeddings:
            embedding.reset_parameters()

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(1)

        out = 0
        for i in range(x.size(1)):
            out += self.embeddings[i](x[:, i])
        return out


class BondEncoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(BondEncoder, self).__init__()

        self.embeddings = torch.nn.ModuleList()

        for i in range(3):
            self.embeddings.append(Embedding(6, hidden_channels))

    def reset_parameters(self):
        for embedding in self.embeddings:
            embedding.reset_parameters()

    def forward(self, edge_attr):
        if edge_attr.dim() == 1:
            edge_attr = edge_attr.unsqueeze(1)

        out = 0
        for i in range(edge_attr.size(1)):
            out += self.embeddings[i](edge_attr[:, i])
        return out

In [16]:
import torch
import torch.nn as nn
from torch_scatter import scatter
import numpy as np
import matplotlib.pyplot as plt
# partial scatter func
from functools import partial


class Diff_Hist(nn.Module):
    def __init__(self, centers=[-5, -2.5, 0, 2.5, 5], scale=2):
        super(Diff_Hist, self).__init__()
        """ centers: center of the bins used. 
            scale  : exponential scale for sigmoid. the higher,
                the lower the amplitud of the bins.
        """
        # save params
        self.scale        = scale
        self.true_centers = centers
        
        # augmenting sigmoid so higher gradient and localized feats
        self.max_scaler   = 1
        self.sigmoid_x    = lambda y: 1/(1+torch.exp(-self.scale*y))
        self.max_scaler   = torch.max(self.func(torch.arange(start=-1, end=1, step=1e-3)))
        self.sigmoid_x    = lambda y: 1/(1+torch.exp(-self.scale*y))
        
        # our function is centered @ 0.5 by default, but not at extremes (then it's centered @ 0)
        self.centers      = [centers[0]]+list(0.5 + np.array(centers[1:-1]))+[centers[-1]]
        self.n_centers    = len(self.centers)

    def forward(self, x, reduce="mean"):
        r""" reduces dim=0 as a set of bins. designed for 2d outs. """
        x_shape = x.shape
        out     = torch.zeros(x_shape[0], self.n_centers, *x.shape[1:], device=device)
        for i, center in enumerate(self.centers):
            # add the 2 extremes to differentiate btween them (+/- and right/left is inversed)
            if i == 0:
                out[:, i] = self.make_right_extreme(x, center)
            elif i == self.n_centers-1:
                out[:, i] = self.make_left_extreme(x, center)
            else:
                out[:, i] = self.make_bin(x, center)
            
        # summarizes a set of (n_examples, n_feats) into a single (n_centers, n_feats)
        if reduce == "sum" or reduce == "add":
            return out.sum(dim=0)
        elif reduce:
            return out.mean(dim=0)
        return out

    def make_bin(self, x, center):
        return self.func(x+center)
    
    def make_left_extreme(self, x, center):
        return (1-self.sigmoid_x(x+center))
    
    def make_right_extreme(self, x, center):
        return self.sigmoid_x(x+center)
        
    def func(self, x):
        # perform minmax scaling so max is 1 and min (is already) 0
        return (self.sigmoid_x(x)-self.sigmoid_x(x-1))/self.max_scaler

    def __repr__(self):
        return "Simple differentiable histogram layer:" + \
               str({"centers"   : self.true_centers,
                    "exp_scale" : self.scale})
    

class Readout_Hist(nn.Module):
    def __init__(self, centers, reduce=None):
        super(Readout_Hist, self).__init__()
        # save centers
        self.true_centers  = centers
        self.n_centers     = len(centers)
        self.diff_hist     = Diff_Hist(centers=centers)
        # reduces nodes -> graph by gather/scatter
        if reduce is None:
            reduce = partial(scatter, reduce="mean")
        self.reduce_scheme = reduce

    def forward(self, x, batch=None, bsize=None, dim=0):
        r""" reduces dim=0 by gather/scatter each node. designed for 2d outs. """
        hist_aux = self.diff_hist(x, reduce=None)
        hist_aux = hist_aux.reshape(hist_aux.shape[0], -1)
        return self.reduce_scheme(hist_aux, batch, dim_size=bsize, dim=dim)

    def __repr__(self):
        return "Readout by histogram:" + str({"centers": self.true_centers})

In [36]:
class Net(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers, dropout=0.0,
                 inter_message_passing=True):
        super(Net, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.inter_message_passing = inter_message_passing
        print("using inter_message_passing:", inter_message_passing)

        self.atom_encoder = AtomEncoder(hidden_channels)
        self.clique_encoder = Embedding(4, hidden_channels)

        self.bond_encoders = ModuleList()
        self.atom_convs = ModuleList()
        self.atom_batch_norms = ModuleList()
        
        # DIFFERENTIABLE HISTOGRAMS
        offset, n   = 2, 7 # from -(2*5) to 2*5 
        centers     = list(range(-offset*n, offset*n+1, offset))
        self.reader = Readout_Hist(centers=centers)

        for _ in range(num_layers):
            self.bond_encoders.append(BondEncoder(hidden_channels))
            nn = Sequential(
                Linear(hidden_channels, 2 * hidden_channels),
                BatchNorm1d(2 * hidden_channels),
                ReLU(),
                Linear(2 * hidden_channels, hidden_channels),
            )
            self.atom_convs.append(GINEConv(nn, train_eps=True))
            self.atom_batch_norms.append(BatchNorm1d(hidden_channels))

        self.clique_convs = ModuleList()
        self.clique_batch_norms = ModuleList()

        for _ in range(num_layers):
            nn = Sequential(
                Linear(hidden_channels, 2 * hidden_channels),
                BatchNorm1d(2 * hidden_channels),
                ReLU(),
                Linear(2 * hidden_channels, hidden_channels),
            )
            self.clique_convs.append(GINConv(nn, train_eps=True))
            self.clique_batch_norms.append(BatchNorm1d(hidden_channels))

        self.atom2clique_lins = ModuleList()
        self.clique2atom_lins = ModuleList()

        for _ in range(num_layers):
            self.atom2clique_lins.append(
                Linear(hidden_channels, hidden_channels))
            self.clique2atom_lins.append(
                Linear(hidden_channels, hidden_channels))

        self.atom_lin = Linear(hidden_channels*len(centers), hidden_channels*len(centers))
        self.clique_lin = Linear(hidden_channels*len(centers), hidden_channels*len(centers))
        # final layer merges info from readout + clique
        self.lin = Linear(hidden_channels*len(centers), out_channels)

    def reset_parameters(self):
        self.atom_encoder.reset_parameters()
        self.clique_encoder.reset_parameters()

        for emb, conv, batch_norm in zip(self.bond_encoders, self.atom_convs,
                                         self.atom_batch_norms):
            emb.reset_parameters()
            conv.reset_parameters()
            batch_norm.reset_parameters()

        for conv, batch_norm in zip(self.clique_convs,
                                    self.clique_batch_norms):
            conv.reset_parameters()
            batch_norm.reset_parameters()

        for lin1, lin2 in zip(self.atom2clique_lins, self.clique2atom_lins):
            lin1.reset_parameters()
            lin2.reset_parameters()

        self.atom_lin.reset_parameters()
        self.clique_lin.reset_parameters()
        self.lin.reset_parameters()
        # self.reader.reset_parameters()

    def forward(self, data):
        x = self.atom_encoder(data.x.squeeze())

        if self.inter_message_passing:
            x_clique = self.clique_encoder(data.x_clique.squeeze())

        for i in range(self.num_layers):
            edge_attr = self.bond_encoders[i](data.edge_attr)
            x = self.atom_convs[i](x, data.edge_index, edge_attr)
            x = self.atom_batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, training=self.training)

            if self.inter_message_passing:
                row, col = data.atom2clique_index

                x_clique = x_clique + F.relu(self.atom2clique_lins[i](scatter(
                    x[row], col, dim=0, dim_size=x_clique.size(0),
                    reduce='mean')))

                x_clique = self.clique_convs[i](x_clique, data.tree_edge_index)
                x_clique = self.clique_batch_norms[i](x_clique)
                x_clique = F.relu(x_clique)
                x_clique = F.dropout(x_clique, self.dropout,
                                     training=self.training)

                x = x + F.relu(self.clique2atom_lins[i](scatter(
                    x_clique[col], row, dim=0, dim_size=x.size(0),
                    reduce='mean')))
        # print(x.shape, "before readout")
        x = self.reader(x, data.batch)
        # print(x.shape, "after readout")
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.atom_lin(x)
        # print(x.shape, "after atom_lin")
        

        if self.inter_message_passing:
            tree_batch = torch.repeat_interleave(data.num_cliques)
            # x_clique = scatter(x_clique, tree_batch, dim=0, dim_size=x.size(0), reduce='mean')
            x_clique = self.reader(x_clique, tree_batch, bsize=x.size(0))
            x_clique = F.dropout(x_clique, self.dropout,
                                 training=self.training)
            x_clique = self.clique_lin(x_clique)
            x = x + x_clique

        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        # print(x.shape, "before last")
        x = self.lin(x)
        return x

In [37]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import tree_decomposition

from rdkit import Chem
from rdkit.Chem.rdchem import BondType

bonds = [BondType.SINGLE, BondType.DOUBLE, BondType.TRIPLE, BondType.AROMATIC]


def mol_from_data(data):
    mol = Chem.RWMol()

    x = data.x if data.x.dim() == 1 else data.x[:, 0]
    for z in x.tolist():
        mol.AddAtom(Chem.Atom(z))

    row, col = data.edge_index
    mask = row < col
    row, col = row[mask].tolist(), col[mask].tolist()

    bond_type = data.edge_attr
    bond_type = bond_type if bond_type.dim() == 1 else bond_type[:, 0]
    bond_type = bond_type[mask].tolist()

    for i, j, bond in zip(row, col, bond_type):
        assert bond >= 1 and bond <= 4
        mol.AddBond(i, j, bonds[bond - 1])

    return mol.GetMol()


class JunctionTreeData(Data):
    def __inc__(self, key, item):
        if key == 'tree_edge_index':
            return self.x_clique.size(0)
        elif key == 'atom2clique_index':
            return torch.tensor([[self.x.size(0)], [self.x_clique.size(0)]])
        else:
            return super(JunctionTreeData, self).__inc__(key, item)


class JunctionTree(object):
    def __call__(self, data):
        mol = mol_from_data(data)
        out = tree_decomposition(mol, return_vocab=True)
        tree_edge_index, atom2clique_index, num_cliques, x_clique = out

        data = JunctionTreeData(**{k: v for k, v in data})

        data.tree_edge_index = tree_edge_index
        data.atom2clique_index = atom2clique_index
        data.num_cliques = num_cliques
        data.x_clique = x_clique

        return data

In [38]:
# edit the function causing the error: add argument chem=None + modify function code: 
# Chem=chem if chem is not None else Chem

# tree_decomposition(Chem.MolFromSmiles("cicccc1c"), return_vocab=True)
# once modified and saved, restart the environmnet, comment this cell and run the experiment.

In [39]:
import argparse

import torch
from torch.optim import Adam
import numpy as np
from sklearn.metrics import roc_auc_score

from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.data import DataLoader
from torch_geometric.transforms import Compose

class Argparse_emulate():
    def __init__(self, device=0, hidden_channels=256, num_layers=2, dropout=0.5,
               epochs=100, no_inter_message_passing="store_true"):
        self.device = device
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers
        self.dropout = 0.6 # dropout
        self.epochs = epochs
        self.no_inter_message_passing = no_inter_message_passing
        return

args = Argparse_emulate()
# parser.add_argument('--device', type=int, default=0)
# parser.add_argument('--hidden_channels', type=int, default=256)
# parser.add_argument('--num_layers', type=int, default=2)
# parser.add_argument('--dropout', type=float, default=0.5)
# parser.add_argument('--epochs', type=int, default=100)
# parser.add_argument('--no_inter_message_passing', action='store_true')
# args = parser.parse_args()
# print(args)


class OGBTransform(object):
    # OGB saves atom and bond types zero-index based. We need to revert that.
    def __call__(self, data):
        data.x[:, 0] += 1
        data.edge_attr[:, 0] += 1
        return data


transform = Compose([OGBTransform(), JunctionTree()])

name = 'ogbg-molhiv'
dataset = PygGraphPropPredDataset(name, 'data', pre_transform=transform)

# correct splits
split_idx = dataset.get_idx_split()
train_dataset = dataset[split_idx['train']]
val_dataset = dataset[split_idx['valid']]
test_dataset = dataset[split_idx['test']]

train_loader = DataLoader(train_dataset, 128, shuffle=True)
val_loader = DataLoader(val_dataset, 128, shuffle=False)
test_loader = DataLoader(test_dataset, 128, shuffle=False)


device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'

In [ ]:
def train(epoch, vals=False):
    values = []
    model.train()

    total_loss = 0
    for i,data in tqdm(enumerate(train_loader)):
        data = data.to(device)
        optimizer.zero_grad()
        mask = ~torch.isnan(data.y)
        out = model(data)[mask]
        y = data.y.to(torch.float)[mask]
        loss = torch.nn.BCEWithLogitsLoss()(out, y)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
        # record beta and p values
        if vals:
            # display computational graph
            # global g
            #g = make_dot(out)
            # "a"+9
            pass

    return total_loss / len(train_loader.dataset), values


@torch.no_grad()
def test(loader):
    model.eval()

    y_preds, y_trues = [], []
    for data in loader:
        data = data.to(device)
        y_preds.append(model(data))
        y_trues.append(data.y)

    y_pred = torch.cat(y_preds, dim=0).cpu().numpy()
    y_true = torch.cat(y_trues, dim=0).cpu().numpy()

    rocauc_list = []
    for i in range(y_true.shape[1]):
        # AUC is only defined when there is at least one positive data.
        if np.sum(y_true[:, i] == 1) > 0 and np.sum(y_true[:, i] == 0) > 0:
            # ignore nan values
            is_labeled = y_true[:, i] == y_true[:, i]
            rocauc_list.append(
                roc_auc_score(y_true[is_labeled, i], y_pred[is_labeled, i]))

    return {"rocauc": sum(rocauc_list) / len(rocauc_list)}


values     = []
test_perfs = []
for run in range(10):
    print()
    print(f'Run {run}:')
    print()
    model = Net(hidden_channels=args.hidden_channels,
            out_channels=dataset.num_tasks, num_layers=args.num_layers,
            dropout = args.dropout if run<10 else 0.6, # edited to increase dropout
            inter_message_passing= args.no_inter_message_passing).to(device) # not args.no_inter

    model.reset_parameters()
    optimizer = Adam(model.parameters(), lr=0.0001)

    best_val_perf = test_perf = 0
    for epoch in range(1, args.epochs + 1):
        loss, epoch_values = train(epoch, vals=True)
        train_perf = test(train_loader)
        val_perf = test(val_loader)

        if val_perf["rocauc"] > best_val_perf:
            best_val_perf = val_perf["rocauc"]
            test_perf = test(test_loader)

        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_perf["rocauc"]:.4f}, Val: {val_perf["rocauc"]:.4f}, '
              f'Test: {test_perf["rocauc"]:.4f}')
        if epoch % 10 == 0:
            print("Recorded values:", epoch_values)

    test_perfs.append(test_perf["rocauc"])
    # values.append(epoch_values[-1])

test_perf = torch.tensor(test_perfs)
print('===========================')
print(f'Final Test: {test_perf.mean():.4f} ± {test_perf.std():.4f}')


Run 0:

using inter_message_passing: store_true


258it [00:37,  6.88it/s]
1it [00:00,  6.85it/s]

Epoch: 001, Loss: 0.1669, Train: 0.6953, Val: 0.6703, Test: 0.6531


258it [00:37,  6.90it/s]
1it [00:00,  6.41it/s]

Epoch: 002, Loss: 0.1507, Train: 0.7446, Val: 0.7558, Test: 0.7680


...

258it [00:37,  6.80it/s]
1it [00:00,  6.06it/s]

Epoch: 007, Loss: 0.1309, Train: 0.8001, Val: 0.8048, Test: 0.7715


258it [00:37,  6.81it/s]
1it [00:00,  6.10it/s]

Epoch: 008, Loss: 0.1304, Train: 0.8113, Val: 0.8050, Test: 0.7877


258it [00:38,  6.75it/s]


In [29]:
# sum. dropout=0.5. readout-> layer -> +cliques -> final

0it [00:00, ?it/s]


Run 0:

using inter_message_passing: store_true


258it [00:28,  8.93it/s]
1it [00:00,  7.69it/s]

Epoch: 001, Loss: 0.1719, Train: 0.7334, Val: 0.7373, Test: 0.7231


258it [00:28,  9.04it/s]
1it [00:00,  7.46it/s]

...

258it [00:28,  9.00it/s]
1it [00:00,  8.00it/s]

Epoch: 099, Loss: 0.0825, Train: 0.9695, Val: 0.8250, Test: 0.7678


258it [00:28,  9.02it/s]


Epoch: 100, Loss: 0.0796, Train: 0.9701, Val: 0.8207, Test: 0.7678
Recorded values: []


IndexError: list index out of range

### Analize params

In [ ]:
import matplotlib.pyplot as plt
p_epoch    = np.array([batch["p"] for batch in epoch_values]).flatten()
beta_epoch = np.array([batch["beta"] for batch in epoch_values]).flatten() 

plt.figure(figsize=(10,4))
plt.title("Evolution of parameters beta and p during training - Power Mean")
plt.plot(p_epoch, "r-", label="p")
plt.plot(beta_epoch, "b-", label="beta")
plt.ylim(0, 1.1*np.amax(np.maximum(p_epoch, beta_epoch)))
plt.legend()
plt.show()